In [1]:
import pandas as pd
import numpy as np

# Si no existe 'df', intenta leerlo desde 'output_price.csv'
try:
    df  # reference to check existencia
except NameError:
    df = pd.read_csv('output_price.csv')

# Agrega columnas de logaritmo natural para 'precio_promedio' y 'kilos'
cols = [('precio_promedio', 'ln_precio_promedio'), ('kilos', 'ln_kilos')]
for original, nueva in cols:
    if original not in df.columns:
        raise KeyError(f"Falta la columna '{original}' en el DataFrame")
    serie = df[original]
    # Evita logaritmos de valores no positivos devolviendo NaN
    df[nueva] = np.where(serie > 0, np.log(serie), np.nan)

# Vista rápida
df.head()


c:\Users\SebastiánIgnacioCata\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\SebastiánIgnacioCata\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\SebastiánIgnacioCata\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\SebastiánIgnacioCata\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


,semana,categoria,familia,sku,nombre_producto,precio_promedio,kilos,venta,ln_precio_promedio,ln_kilos
0,2024-10-21,ACEITES Y VINAGRES,ACEITES DE OLIVA,104521640,ACEITE OLIVA NUTRICAMPO 5 l,7565.0,1090,8246175,8.931288,6.993933
1,2024-10-21,ACEITES Y VINAGRES,ACEITES DE OLIVA,104524420,ACEITE OLIVA E/VIRGEN ZAYTA 500 ml,9339.0,2,14009,9.141954,0.693147
2,2024-10-21,ACEITES Y VINAGRES,ACEITES DE OLIVA,104537630,ACEITE OLIVA NUTRICAMPO 500 ML,8354.0,7,58480,9.030496,1.945910
3,2024-10-21,ACEITES Y VINAGRES,ACEITES DE OLIVA,104539450,ACEITE OLIVA E/VIRGEN OLIVE&CO 250 ml,18141.0,14,244908,9.805930,2.639057
4,2024-10-21,ACEITES Y VINAGRES,ACEITES DE OLIVA,104539460,"ACEITE OLIVA E/VIRGEN OLIVE&CO 4,85 L",13674.0,175,2387403,9.523251,5.164786


In [2]:
# Ajuste log-log por SKU: ln(Q) = alpha + beta ln(P) + e
# Usa OLS simple con numpy (sin dependencias externas).

# Asegura columnas requeridas
required = ['sku', 'ln_precio_promedio', 'ln_kilos']
missing = [c for c in required if c not in df.columns]
if missing:
    raise KeyError(f"Faltan columnas en df: {missing}")

# Filtra valores válidos
df_valid = df[required].copy()
df_valid = df_valid.replace([np.inf, -np.inf], np.nan).dropna()

def fit_group(g):
    x = g['ln_precio_promedio'].to_numpy(dtype=float)
    y = g['ln_kilos'].to_numpy(dtype=float)
    n = x.size
    if n < 2 or np.var(x) == 0:
        return pd.Series({'alpha': np.nan, 'beta': np.nan, 'n_puntos': int(n), 'r2': np.nan})
    # Ajuste lineal y = a + b x
    beta, alpha = np.polyfit(x, y, 1)  # devuelve [b, a]
    r = np.corrcoef(x, y)[0, 1]
    r2 = float(r ** 2)
    return pd.Series({'alpha': float(alpha), 'beta': float(beta), 'n_puntos': int(n), 'r2': r2})

res = df_valid.groupby('sku', as_index=False).apply(fit_group).reset_index(drop=True)

# Agrega nombre de producto si está disponible
if 'nombre_producto' in df.columns:
    nombres = df[['sku', 'nombre_producto']].drop_duplicates('sku')
    res = res.merge(nombres, on='sku', how='left')
    cols = ['sku', 'nombre_producto', 'n_puntos', 'beta', 'alpha', 'r2']
    res = res[[c for c in cols if c in res.columns]]

# Guarda CSV
out_csv = 'elasticidades_loglog_por_sku.csv'
res.to_csv(out_csv, index=False)
out_csv, res.head()


c:\Users\SebastiánIgnacioCata\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\numpy\lib\_function_base_impl.py:3065: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\SebastiánIgnacioCata\AppData\Local\Python\pythoncore-3.14-64\Lib\site-packages\numpy\lib\_function_base_impl.py:3066: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
C:\Users\SebastiánIgnacioCata\AppData\Local\Temp\ipykernel_8528\119478118.py:21: RankWarning: Polyfit may be poorly conditioned
  beta, alpha = np.polyfit(x, y, 1)  # devuelve [b, a]
C:\Users\SebastiánIgnacioCata\AppData\Local\Temp\ipykernel_8528\119478118.py:21: RankWarning: Polyfit may be poorly conditioned
  beta, alpha = np.polyfit(x, y, 1)  # devuelve [b, a]
C:\Users\SebastiánIgnacioCata\AppData\Local\Temp\ipykernel_8528\119478118.py:21: RankWarning: Polyfit may be poorly conditioned
  beta, alpha = np.polyfit(x, y, 1)  # devuelve [b, a]
C:\Users\SebastiánIgnacioCata\AppData\Local\Tem

('elasticidades_loglog_por_sku.csv',
          sku                        nombre_producto  n_puntos      beta  \
 0  100093620  JALEA EN POLVO FRAMBUESA CARICIA 1 kg      53.0 -1.539006   
 1  100093710                    MAC 65/A 220V./60HZ       1.0       NaN   
 2  100093730        PRENSA CAFE DINAMOMETRICA 25 kg       1.0       NaN   
 3  100093780    JALEA EN POLVO NARANJA CARICIA 1 kg      53.0 -1.314447   
 4  100093790       JALEA EN POLVO PIÑA CARICIA 1 kg      53.0  4.695485   
 
        alpha        r2  
 0  16.488285  0.002636  
 1        NaN       NaN  
 2        NaN       NaN  
 3  14.620217  0.005461  
 4 -36.390805  0.025824  )

In [ ]:
# Selector de SKU y plot de nube + recta de ajuste
import matplotlib.pyplot as plt

try:
    import ipywidgets as widgets
    from IPython.display import display
    USE_WIDGETS = True
except Exception:
    USE_WIDGETS = False

# Asegura df y res disponibles
if 'df' not in globals():
    df = pd.read_csv('output_price.csv')
    if 'ln_precio_promedio' not in df.columns:
        df['ln_precio_promedio'] = np.where(df['precio_promedio'] > 0, np.log(df['precio_promedio']), np.nan)
    if 'ln_kilos' not in df.columns:
        df['ln_kilos'] = np.where(df['kilos'] > 0, np.log(df['kilos']), np.nan)

if 'res' not in globals():
    try:
        res = pd.read_csv('elasticidades_loglog_por_sku.csv')
    except FileNotFoundError:
        res = pd.DataFrame(columns=['sku','alpha','beta','r2','n_puntos'])

def plot_sku(selected_sku):
    skustr = str(selected_sku)
    datos = df[df['sku'].astype(str) == skustr][['ln_precio_promedio','ln_kilos']].replace([np.inf, -np.inf], np.nan).dropna()
    fila = res[res['sku'].astype(str) == skustr]
    alpha = float(fila['alpha'].iloc[0]) if (not fila.empty and 'alpha' in fila) else np.nan
    beta  = float(fila['beta'].iloc[0])  if (not fila.empty and 'beta'  in fila) else np.nan
    r2    = float(fila['r2'].iloc[0])    if (not fila.empty and 'r2'    in fila) else np.nan

    nombre = None
    if 'nombre_producto' in df.columns:
        names = df.loc[df['sku'].astype(str) == skustr, 'nombre_producto']
        if not names.empty:
            nombre = str(names.iloc[0])

    plt.figure(figsize=(7,5))
    x = datos['ln_precio_promedio'].to_numpy()
    y = datos['ln_kilos'].to_numpy()
    plt.scatter(x, y, alpha=0.75, label='Datos (ln P vs ln Q)')

    if x.size > 0 and not (np.isnan(alpha) or np.isnan(beta)):
        xx = np.linspace(np.nanmin(x), np.nanmax(x), 100)
        yy = alpha + beta * xx
        plt.plot(xx, yy, color='red', label=f'Recta: y = {alpha:.2f} + {beta:.2f} x')

    plt.xlabel('ln(P) precio_promedio')
    plt.ylabel('ln(Q) kilos')
    titulo = f'SKU {skustr}'
    if nombre:
        titulo += f' - {nombre}'
    plt.title(titulo)

    if not np.isnan(r2):
        xmin, xmax = plt.xlim(); ymin, ymax = plt.ylim()
        plt.text(xmin + 0.02*(xmax-xmin), ymax - 0.08*(ymax-ymin), f'R² = {r2:.3f}',
                 fontsize=11, bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

    plt.grid(True, alpha=0.25)
    plt.legend()
    plt.show()

# Construye opciones de SKU
if 'sku' in df.columns:
    skus = sorted(df['sku'].astype(str).unique().tolist())
else:
    skus = sorted(res['sku'].astype(str).unique().tolist())

if USE_WIDGETS and skus:
    opciones = []
    if 'nombre_producto' in df.columns:
        for s in skus:
            nm = df.loc[df['sku'].astype(str) == s, 'nombre_producto']
            etiqueta = f"{s} - {nm.iloc[0]}" if not nm.empty else s
            opciones.append((etiqueta, s))
    else:
        opciones = [(s, s) for s in skus]
    dd = widgets.Dropdown(options=opciones, description='SKU:', layout=widgets.Layout(width='70%'))
    out = widgets.interactive_output(plot_sku, {'selected_sku': dd})
    display(widgets.VBox([dd, out]))
else:
    # Fallback: plotea el primer SKU disponible
    if skus:
        plot_sku(skus[0])
    else:
        print('No hay SKUs disponibles para graficar.')
